
# 📘 4강 — 함수 합성과 파이프라인 (Function Composition & Pipeline)

### 🎯 학습목표
- 작은 순수 함수들을 **합성(Composition)** 하여 안정적인 큰 동작을 만든다.
- 읽기 흐름이 좋은 **파이프라인(Pipeline)** 으로 데이터 변환을 단계화한다.
- **지연 계산(Iterables/Generators)** 과 결합해 **메모리 효율 처리**를 익힌다.

### 🧩 핵심개념
- 합성: `compose(f, g, h)(x) == f(g(h(x)))` (오→왼)
- 파이프: `pipe(x, f, g, h) == h(g(f(x)))` (왼→오)
- 순수성, 불변성, 재사용/테스트 용이성, 레이지 map/filter/take

### 📝 키워드
`compose`, `pipe`, `map`, `filter`, `take`, `generator`, `partial`, `reduce`, `immutable`, `lazy`

---
간단 예시: 숫자에 **두 배 → 1 더하기 → 제곱**을 순차 적용하는 흐름을 합성/파이프 두 방식으로 표현한다.


In [4]:

from __future__ import annotations
from collections.abc import Callable
from typing import Any

def compose(*funcs: Callable[[Any], Any]) -> Callable[[Any], Any]:
    """오른쪽→왼쪽 합성: compose(f, g, h)(x) == f(g(h(x)))."""
    def _composed(x: Any) -> Any:
        for f in reversed(funcs):
            x = f(x)
        return x
    return _composed

def pipe(x: Any, *funcs: Callable[[Any], Any]) -> Any:
    """왼쪽→오른쪽 파이프: pipe(x, f, g, h) == h(g(f(x)))."""
    for f in funcs:
        x = f(x)
    return x

# 데모용 작은 함수들
inc    = lambda x: x + 1
double = lambda x: x * 2
square = lambda x: x * x

# 합성 (오른쪽→왼쪽)
combo = compose(square, inc, double)  # square(inc(double(x)))
print("compose:", combo(3))  # 기대: 49

# 파이프 (왼쪽→오른쪽)
print("pipe   :", pipe(3, double, inc, square))  # 기대: 49

# 문자열 파이프 예시
strip = str.strip
lower = str.lower
def collapse_spaces(s: str) -> str:
    return " ".join(s.split())

print("text   :", pipe("  HeLLo   WORLD  ", strip, lower, collapse_spaces))  # 기대: 'hello world'


compose: 49
pipe   : 49
text   : hello world



## 🔧 주요 기능 설명

- **`compose(*funcs)`**: 수학적 함수 합성 표기와 동일하게 **오른쪽→왼쪽**으로 적용되어 추론/검증에 유리합니다.
- **`pipe(x, *funcs)`**: 데이터 흐름을 **왼쪽→오른쪽**으로 보여주어 단계적 로깅/디버깅에 좋습니다.
- **레이지 유틸**: `map_(fn, it)`, `filter_(pred, it)`, `take(n, it)`을 사용해 **메모리 효율적** 파이프라인을 구성합니다.
- 타입 힌트: 가변 길이 합성의 정적 타입 추론은 한계가 있으므로, 실무에서는 **작은 블록 합성**을 권장합니다.


In [5]:

from collections.abc import Iterable, Iterator, Callable
from typing import TypeVar

T = TypeVar("T")
U = TypeVar("U")

def map_(fn: Callable[[T], U], it: Iterable[T]) -> Iterator[U]:
    for x in it:
        yield fn(x)

def filter_(pred: Callable[[T], bool], it: Iterable[T]) -> Iterator[T]:
    for x in it:
        if pred(x):
            yield x

def take(n: int, it: Iterable[T]) -> Iterator[T]:
    """앞에서 n개를 지연 산출 (무한 이터러블에도 안전)."""
    if n <= 0:
        return
        yield  # generator로 인식시키기 위한 더미 (실행되지 않음)
    k = 0
    it = iter(it)
    while k < n:
        try:
            yield next(it)
        except StopIteration:
            return
        k += 1

# 파이프라인 예시: [1..10] → 짝수만 → 2배 → 앞에서 3개
data = range(1, 11)
result = list(
    pipe(
        data,
        lambda it: filter_(lambda x: x % 2 == 0, it),
        lambda it: map_(lambda x: x * 2, it),
        lambda it: take(3, it),
    )
)
print("pipeline:", result)  # 기대: [4, 8, 12]


pipeline: [4, 8, 12]



## 🧪 실습 (실행 중심)

아래 **실습 코드** 셀을 그대로 실행하여 출력과 `assert` 검증이 통과하는지 확인하세요.  
필요하면 입력 값을 바꿔가며 다시 실행해 보세요. (코드 작성은 **과제 섹션**에서 진행)

**예상 결과**
- `even_times3_take5_sum(range(1, 11))` ⇒ **90**
- `normalize_and_join(["  Hello ", " WORLD", "", "PyThOn  "])` ⇒ **"hello-world-python"**
- `total_price_with_vat([{"price": 1000, "qty": 2}, {"price": 500, "qty": 3}])` ⇒ **3850.0**


In [7]:

from typing import Iterable

def even_times3_take5_sum(nums: Iterable[int]) -> int:
    return pipe(
        nums,
        lambda it: filter_(lambda x: x % 2 == 0, it),
        lambda it: map_(lambda x: x * 3, it),
        lambda it: take(5, it),
        lambda it: sum(it),
    )

def normalize_and_join(words: Iterable[str]) -> str:
    def _norm(s: str) -> str:
        return " ".join(str(s).strip().lower().split())
    cleaned = pipe(
        words,
        lambda it: map_(_norm, it),
        lambda it: filter_(lambda s: s != "", it),
        lambda it: list(it),
    )
    return "-".join(cleaned)

def total_price_with_vat(items: Iterable[dict[str, float]]) -> float:
    subtotal = pipe(
        items,
        lambda it: map_(lambda d: float(d.get("price", 0.0)) * float(d.get("qty", 0.0)), it),
        lambda it: sum(it),
    )
    return subtotal * 1.10

# === 실행 & 검증 ===
from pytest import approx

print("▶ 실행: even_times3_take5_sum(range(1, 11))")
r1 = even_times3_take5_sum(range(1, 11))
print("출력:", r1)
assert r1 == 90, f"기대값 90, 실제 {r1}"

print("\n▶ 실행: normalize_and_join(['  Hello ', ' WORLD', '', 'PyThOn  '])")
r2 = normalize_and_join(["  Hello ", " WORLD", "", "PyThOn  "])
print("출력:", r2)
assert r2 == "hello-world-python", f"기대값 'hello-world-python', 실제 {r2!r}"

print("\n▶ 실행: total_price_with_vat(...)")
r3 = total_price_with_vat([{"price": 1000, "qty": 2}, {"price": 500, "qty": 3}])
print("출력:", r3)
assert r3 == approx(3850.0, rel=1e-9, abs=0.0), f"기대값 3850.0, 실제 {r3}"

print("\n✅ 모든 실습 검증 통과! (assert OK)")


▶ 실행: even_times3_take5_sum(range(1, 11))
출력: 90

▶ 실행: normalize_and_join(['  Hello ', ' WORLD', '', 'PyThOn  '])
출력: hello-world-python

▶ 실행: total_price_with_vat(...)
출력: 3850.0000000000005

✅ 모든 실습 검증 통과! (assert OK)



## ⚡ 정리
- **합성**은 오→왼, **파이프**는 왼→오로 읽는다.
- 작은 순수 함수들을 연결하면 **가독성/재사용성/테스트 용이성**이 향상된다.
- 제너레이터(map_/filter_/take)와 결합해 **메모리 효율적인 레이지 파이프라인**을 만들 수 있다.
- 실제 코드에서는 **작은 합성 단위**(2~3개)를 층층이 쌓는 전략이 타입과 디버깅에 유리하다.

## 📝 Assignment
(1) `compose2`를 구현하라: `f: (B→C)`, `g: (A→B)`를 받아 `(A→C)`를 반환하는 타입 안전 합성 함수를 작성하고 예제를 통해 검증하라.  
예) `compose2(str, hex)`로 `int -> str` 변환기를 만든 뒤 샘플 입력을 체크.

(2) `pipe_iter`를 구현하라: 입력이 이터러블이면 각 단계를 **Iterable→Iterable** 형태로 강제해 레이지 파이프라인을 유지해야 한다.  
힌트: 각 단계 시그니처를 `Callable[[Iterable[T]], Iterable[U]]`로 맞추고, `yield from`을 적극 활용하라.

(3) 미니 프로젝트 — 주문 목록 처리  
입력: `{'name': str, 'price': float, 'qty': int, 'category': str}` 시퀀스  
요구: (a) `category='book'` 필터 → (b) `qty>=2` 선택 → (c) `price*qty` 합계에 **10% 할인** 적용 → (d) 소수점 둘째자리 반올림 후 반환.  
반드시 **합성/파이프라인**으로 구성하고, 중간 단계를 분리된 순수 함수로 둘 것.

## 규칙
- for/while 대신 **map/filter/generator** 우선. (필요 시 `sum`, `any`, `all` 등 내장 사용 가능)
- 모든 함수는 **입력을 변경하지 말 것(불변성)**.
- I/O(파일/네트워크/print) 없는 **순수 함수**로 작성. (디버그 로그는 주석 처리)
- 함수/변수에 **정확한 타입 힌트**를 달 것.
